In [1]:
## Use NBviewer for better rendering ----------------------------->S
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()"><button>Toggle Code</button></a>
''')

<h1><center>Generative Adversarial Nets</center></h1>
<br>
<p><center>Ian J. Goodfellow, Jean Pouget-Abadie, Mehdi Mirza, Bing Xu, David Warde-Farley, Sherjil Ozair, Aaron Courville, Yoshua Bengio</center></p>
<h6><center>Université de Montréal</center></h6>
<br>
<br>
<br>
<br>
<br>
<p style="font-size: 10px;">CMI Lab Journal Club, 02/18/2019, Andrew Van</p>
<script type="text/javascript">
$(window).load(function(){
    Reveal.configure({
        transition: 'fade' // none/fade/slide/convex/concave/zoom
    })
});    
</script>

<h3><center>The motivation for GANs</center></h3>

- They can make really convincing fake images!

<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
<img src="images/stylegan-teaser.png" width=100% height=auto>
</div>
<p><center>These are not real people! <a href="https://arxiv.org/abs/1812.04948" target="_blank">StyleGAN</a></center></p>

<h3><center>The motivation for GANs</center></h3>

<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
<img src="images/superresolution.png" width=100% height=auto>
</div>
<p><center><a href="https://arxiv.org/abs/1609.04802" target="_blank">Super-Resolution</a></center></p>

<h3><center>The motivation for GANs</center></h3>

<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
<img src="images/highresimgsynth.gif" width=100% height=auto>
</div>
<p><center><a href="https://arxiv.org/abs/1711.11585" target="_blank">High-resolution image synthesis</a></center></p>

<h3><center>The motivation for GANs</center></h3>

<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
<img src="images/imageinpainting.png" width=100% height=auto>
</div>
<p><center><a href="https://arxiv.org/abs/1604.07379" target="_blank">Image Inpainting</a></center></p>

<h3><center>The motivation for GANs</center></h3>

<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
<img style="display:block;margin:auto;" src="images/tumor.png" width=75% height=auto>
</div>
<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
<img style="display:block;margin:auto;" src="images/tumor2.png" width=75% height=auto>
</div>
<p><center><a href="https://arxiv.org/abs/1703.05921" target="_blank">Anomaly Detection (Tumor Detection)</a></center></p>

<h3><center>Overview</center></h3>

- Generative Modeling
- Adversarial Training
- Theoretical guarantees of optimality/convergence
- Results/Discussion

<h3><center>What is generative modeling?</center></h3>

- Most successes in deep learning so far have been discriminative models
    - Mapping high-dimensional data to a class label
    - e.g. Given an image, is it a cat/dog?
- Generative models answer a somewhat opposite question
    - Given a sample drawn from a distribution, we want to learn an estimate of that distribution
    - e.g. Given a set of cats/dogs, learn the images that generate each class.
    
<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
<img src="images/density_estimation.png" width=100% height=auto>
</div>
<p><center>Generative Modeling: Estimating a distribution from a sample</center></p>

<h3><center>Generative modeling from the framework of Maximum Likelihood</center></h3>

- We want to estimate a distribution $p_{data}$, from the empirical distribution $\hat{p}_{data}$ (our training data/sample).
- Define a model, $p_{model}({\bf x}; \theta)$ parameterized by $\theta$.
- We want $p_{model}$ to estimate $p_{data}$, using samples, ${\bf x}^{(i)}$, drawn from $\hat{p}_{data}$
- We want to solve:
$$\theta^{*} = \underset{\theta}{arg\,max} \sum_{i=1}^{m} \log{p_{model}({\bf x}^{(i)}; \theta)}$$
- What is our choice of $p_{model}$? and how do we maximize it?

<h3><center>Taxonomy of generative modeling</center></h3>

<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
    <img style="display:block;margin:auto;" src="images/taxonomy.png" width=80% height=auto>
</div>
<p><center>Types of generative models (maximum likelihood)</center></p>

<h3><center>Properties of GANs compared to other models</center></h3>

- GANs are implicit density models
    - A black box: we don't have access to the actual distribution, we can only draw samples from it.
- Explicit models define the distribution explicitly. We have a known representation of $p_{model}$.
    - The difficulty comes in having an explicit model complex enough to capture the data complexity of $p_{data}$, while still being computationally tractable.
        - Bound on complexity of model to ensure tractability
    - e.g. Fully Visible Belief Networks (FVBNs), Variational Autoencoders
- Advantages
    - More computationally tractable than explicit models
        - Does not use any Markov Chains
        - Can be trained using back-propagation
    - No approximations needed. Can learn any $p_{data}$ distribution (given enough examples)
    - Subjectively, the research community has found that GANs provide better samples than any other method

<h3><center>Adversarial Training</center></h3>

- There are two neural networks:
    - Generator, $G$
        - To learn the generator's distribution $p_{g}$ over the data ${\bf x}$, define a prior $p_{\bf z}({\bf z})$ on input latent variables, ${\bf z}$
            - ${\bf z}$ is usually chosen to be a uniform random variable
        - We create a mapping from ${\bf z} \rightarrow {\bf x}$ through the generator, $G({\bf z};\theta_{g})$
            - Thus, we obtain "fake" samples from $G$, i.e. ${\bf x} = G({\bf z};\theta_{g})$
    - Discriminator, $D$
        - The discriminator is trained on both the real data, and the data generated from the generator.
        - The goal is to distinguish the real data from the generated fake data
            - This is represented by $D({\bf x};\theta_{d})$, which is the probability that a sample ${\bf x}$ comes from $p_{data}$ rather than $p_{g}$
    - Train both networks with back-propagation/gradient descent

<h3><center>Adversarial Training (continued)</center></h3>

- The two networks are trained in an adversarial manner
    - The objective of the generator network is to fool the discriminator
    - The objective of the discriminator is to distinguish fake samples from real samples
- This adversarial training framework can be described as a two-player minimax game (with value function $V(G,D)$):

    $$ \underset{G}{\min} \underset{D}{\max} V(D,G) = \mathbb{E}_{{\bf x} \sim p_{data}({\bf x})}[\log{D({\bf x})}] + \mathbb{E}_{{\bf z} \sim p_{\bf z}({\bf z})}[\log{1 - D(G({\bf z}))}] $$

- One can use game theory to analyze this framework, and can find that the Nash equilibrium of this game occurs when $p_{g} = p_{data}$ (more on this in a bit)
- Note that in practice, most people tend to train G to maximize $\log{D(G({\bf z}))}$, instead of minimizing $\log{1 - D(G({\bf z}))}$ (provides stronger gradients)

<img src="images/training.png" width=100% height=auto>

<h3><center>Adversarial Training (continued)</center></h3>

- (a) Adversarial pair near convergence
- (b) For any given $G$, $D$ converges to $\frac{p_{data}({\bf x})}{p_{data}({\bf x}) + p_{g}({\bf x})}$, the optimal discriminator
- (c) After an update to $G$, gradient of $D$ guides $G$ to flow to regions classified as data
- (d) If $G$ and $D$ have enough network capacity, $p_{g}$ converges to $p_{data}$ and the discriminator cannot tell the difference between the fake and real samples, i.e. $D({\bf x}) = \frac{1}{2}$ 

<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
    <img src="images/adversarial.png" width=100% height=auto>
</div>
<p><center>(D, blue, dashed line), (G, green, solid line), (True data distribution, black, dotted line)</center></p>

<h3><center>Global Optimality of $p_{g} = p_{data}$</center></h3>

- **Proposition 1.** For $G$ fixed, the optimal discriminator $D$ is:

    $$D_{G}^{*}({\bf x}) = \frac{p_{data}({\bf x})}{p_{data}({\bf x}) + p_{g}({\bf x})}$$

&nbsp;&nbsp;&nbsp;**Proof**:
- For any fixed $G$, $D$ tries to maximize the quantity $V(G,D)$:

    $$ V(D,G) = \mathbb{E}_{{\bf x} \sim p_{data}({\bf x})}[\log{D({\bf x})}] + \mathbb{E}_{{\bf z} \sim p_{\bf z}({\bf z})}[\log{1 - D(G({\bf z}))}] $$

- We can remove dependence on G, by replacing the expectation with respect to ${\bf z}$ to one with respect to ${\bf x}$, we get:

    $$ V(D) = \mathbb{E}_{{\bf x} \sim p_{data}({\bf x})}[\log{D({\bf x})}] + \mathbb{E}_{{\bf x} \sim p_{g}({\bf x})}[\log{1 - D({\bf x})}] $$

- Taking the derivative of this term and setting to 0 gives **Proposition 1** defined above

<h3><center>Global Optimality of $p_{g} = p_{data}$ (continued)</center></h3>

- We can reformulate the minimax game as:

$$ C(G) = \mathbb{E}_{{\bf x} \sim p_{data}({\bf x})}[\log{\frac{p_{data}({\bf x})}{p_{data}({\bf x}) + p_{g}({\bf x})}}] + \mathbb{E}_{{\bf x} \sim p_{g}({\bf x})}[\log{\frac{p_{g}({\bf x})}{p_{data}({\bf x}) + p_{g}({\bf x})}}] $$

- **Theorem 1.** The global minimum of the virtual training criterion $C(G)$ is achieved if and only if $p_{g} = p_{data}$. At the point, $C(G)$ achieves the value $-\log{4}$.

&nbsp;&nbsp;&nbsp;**Proof:**
- If we assume $p_{g} = p_{data}$, $D_{G}^{*}({\bf x}) = \frac{1}{2}$, and we get that $C(G) = -\log 4$
- (With some effort) we can see that this is the best minimum by rearranging $C(G)$:

    $$ C(G) = -\log{4} + KL\left(p_{data} \Big{\|} \frac{p_{data}+p_{g}}{2} \right) + KL\left(p_{g} \Big{\|} \frac{p_{data}+p_{g}}{2} \right)$$
where $KL$ is the Kullback-Leibler divergence.

<h3><center>Global Optimality of $p_{g} = p_{data}$ (continued)</center></h3>

-  We recognize that the second half of the expression is twice the Jensen-Shannon divergence, which measures the similarity between two probability distribution:

    $$ C(G) = -\log{4} + 2\cdot JSD(p_{data} \| p_{g}) $$

- By construction, the JS divergence, is always $\geq 0$, and 0 when the two probability distributions are the same
- It is then easy to see that $-\log 4$ is the global minimum of $C(G)$ and that the only solution is $p_{g} = p_{data}$, i.e. the generator distribution is indistinguishable from the data distribution

<h3><center>Convergence of $p_{g} = p_{data}$</center></h3>

- **Proposition 2.** If $G$ and $D$ have enough capacity, and at each step of Algorithm 1, the discriminator is allowed to reach its optimum given $G$, and $p_{g}$ is updated so as to improve the criterion

    $$ \mathbb{E}_{{\bf x} \sim p_{data}({\bf x})}[\log{D_{G}^{*}({\bf x})}] + \mathbb{E}_{{\bf x} \sim p_{g}({\bf x})}[\log{1 - D_{G}^{*}({\bf x})}] $$
then $p_{g}$ converges to $p_{data}$

&nbsp;&nbsp;&nbsp;**Proof**:
- Consider $V(G,D)$ = $U(p_{g},D)$. $U(p_{g},D)$ is convex in $p_{g}$.
    - “The subderivatives of a supremum of convex functions include the derivative of the function at the point where the maximum is attained.” This [post](https://medium.com/@samramasinghe/generative-adversarial-networks-a-theoretical-walk-through-5889d5a8f2bb) may help understand this statement.
- With sufficiently small updates of $p_{g}$, $p_{g}$ converges to $p_{x}$
- In practice, we optimize the $\theta_{g}$, the parameters of the network $G$, rather than $p_{g}$ itself
    - Performance of neural networks still hold up well even without theoretical guarantees.

<h3><center>Architecture and Results</center></h3>

- GAN architecture
    - Generator: Mixture of ReLU and Sigmoid Activations
    - Discriminator: Maxout activations and Dropout
    - Noise only applied as input to bottommost layer of generator network
- Peformance compared with Deep Belief Networks, Stacked Autoencoders, and Generative Stochastic Networks
    - Evaluation done via Parzen window-based log-likelihood estimates on generated samples (higher is better, increased likelihood that generated model fits)
    - Cross-validation done to obtain $\sigma$
    - Goodfellow mentions that this method of evaluation has high variance, but best method to their knowledge (at the time)

<div style="width:fit-content;display:block;margin-right:auto;margin-left:auto;">
    <img src="images/results.png" width=100% height=auto>
</div>
<p><center>GAN Results on MNIST, Toronto Face Database, and CIFAR-10</center></p>

<img src="images/results2.png" width=100% height=auto>

<img src="images/latentwalk.png" width=100% height=auto>
<p><center>Latent Space Walk</center></p>

<h3><center>Advantages/Disadvantages</center></h3>

- Disadvantages
    - An implicit representation of $p_{g}$
    - $D$ must be synchronized well with $G$ during training
        - G must not be trained too much without updating D, in order to avoid "the Helvetica scenario" a.k.a mode collapse
        - G collapses too many values of ${\bf z}$ to the same ${\bf x}$ to have enough diversity to model $p_{data}$
- Advantages
    - No Markov Chains
    - Back-propogation to get gradients
    - No inferences needed during learning
    - Adaptable model

<h3><center>Proposed Future Directions</center></h3>

- Conditional Generative Models ([Conditional GANs](https://arxiv.org/abs/1411.1784))
- Learned approximate inference ([Recovery of latent variables from GANs](https://arxiv.org/abs/1702.04782))
- Semi-supervised Learning ([Improved Techniques for Training GANs](https://arxiv.org/abs/1606.03498))
- Efficiency improvements (Too many to list here...)

<h1><center>References</center></h1>

1. I. J. Goodfellow et al., “Generative Adversarial Networks,” arXiv:1406.2661 [cs, stat], Jun. 2014.
2. I. Goodfellow, “NIPS 2016 Tutorial: Generative Adversarial Networks,” arXiv:1701.00160 [cs], Dec. 2016.
3. https://srome.github.io/An-Annotated-Proof-of-Generative-Adversarial-Networks-with-Implementation-Notes/